---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

In [15]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [ ]:
# Case 1:
#    04/20/2009; 04/20/09; 4/20/09; 4/3/09

# Case 2:
#    Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
#    20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
#    Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
#    Feb 2009; Sep 2009; Oct 2010

# Case 3:
#    6/2008; 12/2009
#    2009; 2010

In [16]:
def date_sorter():
    # Read the dataset
    import pandas as pd

    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)

    re_1 = r"\d{1,2}[/-]\d{1,2}[/-]\d{2,4}"
    re_2 = r"(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ \-\.,]+(?:\d{1,2}[\w]*[ \-,]+)?[1|2]\d{3}"
    re_3 = r"(?:\d{1,2}/)?[1|2]\d{3}"

    # Correct misspillings
    df = df.str.replace("Janaury", "January")
    df = df.str.replace("Decemeber", "December")

    # Extract dates
    regex = "((%s)|(%s)|(%s))"%(re_1, re_2, re_3)
    dates = df.str.extract(regex)

    # To Series
    dates = pd.Series(pd.to_datetime(dates.iloc[:,0]))

    # Correct 2071 case:
    from datetime import datetime

    current_year = datetime.now().year
    for i in range(dates.shape[0]):
      if dates[i].year >= current_year:
        dates[i] = pd.Timestamp(year=dates[i].year-100, 
                                month=dates[i].month, day=dates[i].day)

    # Sort the dates
    dates.sort_values(ascending=True, inplace=True)

    return pd.Series(dates.index.values)

In [17]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64